# TF acs-2 screen

In [4]:
#%matplotlib inline
import os
import sys
import tifffile as tiff
import pandas as pd
import matplotlib
# For visualising
#matplotlib.use('MacOSX') 
# For saving
#matplotlib.use('Agg') 
matplotlib.use('MacOSX') #TkAgg
import matplotlib.pyplot as plt

import numpy as np
import itertools as IT
import time
import glob
import skimage
from skimage import io as skio
from skimage import color

import cPickle
import gzip

sys.path.append("./src")
from utilities import *

#plt.interactive(True)
matplotlib.rcParams.update({'font.size': 12})
np.set_printoptions(precision=3)



## Set working directory

In [5]:
os.chdir("/Users/Povilas/Dropbox/Projects/Metformin_TF_acs-2/")
sourceloc = "Users/Povilas/Dropbox/Projects/Metformin_TF_acs-2/"

# os.chdir("/home/pnorv/Dropbox/Projects/Metformin_TF_acs-2/")
# sourceloc = "home/pnorv/Dropbox/Projects/Metformin_TF_acs-2/"

odir="."

## Collect info about files

In [6]:
filedata=[]
header=['Replicate_folder','Folder','File']
filedata.append(header)

allfiles=glob.glob('./*/*/*.tiff')

for fl in allfiles:
    rep,fld,fln=fl.split('/')[1:]
    filedata.append([rep,fld,fln])
    #print rep, fld, fln
    
writecsv(filedata,'Allfiles.csv')

## Parse info

In [8]:
filelist=pd.read_csv('./Allfiles_annotated.csv')

oimages="Images.pkl"
iimagesgz="Images.pkl.gz"
#oimagesgz="Images.pkl.gz"

otbl='All_results_onlyprc.csv'
olabels="Segmentation_onlyprc.pkl"
#olabelsgz="Segmentation__onlyprc.pkl.gz"

res_header=['Replicate_folder', 'Folder', 'File','Replicate','Gene','Type','Worm','W_N','W_Sum','W_Mean','B_N','B_Sum','B_Mean']

loadimages=True

## Load images

In [7]:
if loadimages:
    print time.ctime()
    print "Loading images... {}".format(time.ctime())
    idata = gzip.open(iimagesgz,'rb')
    imagesindex, images = cPickle.load(idata)
    idata.close()
    print "Loading finished! {}".format(time.ctime())
    print time.ctime()
else:
    imagesindex={}
    imagesarray=[]

Fri Mar  2 10:27:34 2018
Loading images... Fri Mar  2 10:27:34 2018
Loading finished! Fri Mar  2 10:28:26 2018
Fri Mar  2 10:28:26 2018


## Segment images

In [13]:

#Multiprocessing segmentation


In [9]:
print images.shape

labelsindex={}
labelsarray=[]
contoursarray=[]
filteredarray=[]

results_data=[]

total=np.float(filelist.shape[0])

for i, row in filelist.iterrows():
    
    rep=row['Replicate']
    repfld=row['Replicate_folder']
    fld=row['Folder']
    fl=row['File']
    gene=row['Gene']
    tp=row['Type']
    
    flfull="./{}/{}/{}".format(repfld,fld,fl)
     #, os.path.isfile(flfull)
    
    if loadimages:
        flid=imagesindex[flfull]
        image_raw=images[flid]
        if np.ndim(image_raw)==2:
            image=color.gray2rgb(image_raw)
            v=image_raw
        else:
            image=image_raw
            v=image[:,:,0]
    else:
        imagesindex[flfull]=i       
        image = tiff.imread(flfull)
        v=image[:,:,0]
        #Save only one layer, as they are redundant
        if np.sum(image[:,:,0]-image[:,:,1])==0:
            imagesarray.append(image[:,:,0])
        else:
            imagesarray.append(image)

    imagef,labeled_worms,contour,worms=labeller4(v)
    
    labelsindex[flfull]=i
    labelsarray.append(labeled_worms)
    contoursarray.append(contour)
    #filteredarray.append(imagef)
                              

    #Data collection step
    #In Velocity background is summed in only one channel!
    b_sz=np.sum(np.isin(labeled_worms,0))
    b_sum=np.sum(image[labeled_worms==0,0])
    b_mean=b_sum/b_sz

    wormind=[win for win in list(np.unique(labeled_worms)) if win!=0]

    for w in wormind:
        w_sz=np.sum(np.isin(labeled_worms, w))
        #In Velocity worm outline is summed in all 3 channels!
        w_sum=np.sum(image[labeled_worms==w,:])
        w_mean=w_sum/w_sz
        results_data.append([repfld,fld,fl,rep,gene,tp,w,w_sz,w_sum,w_mean,b_sz,b_sum,b_mean])

    prc=(i+1)*100/total
    print "{} --- {:.2f}%".format(flfull,prc)


#Save results
results=pd.DataFrame(results_data,columns=res_header)
results.to_csv(otbl)
    
if not loadimages:
    print("Saving images: {}".format(time.ctime()))
    print "Stacking data..."
    images=np.stack(imagesarray, axis=0)
    f = open(oimages, "w")
    #f = gzip.open(oimagesgz, "w")
    print "Writing..."
    cPickle.dump((imagesindex,images), f)
    f.close()
    os.system("gzip {}".format(oimages))
    print("Saving images complete!")
    

    
print("Saving labels: {}".format(time.ctime()))
print "Stacking..."
#filtered=np.stack(filteredarray,axis=0)
labels=np.stack(labelsarray, axis=0).astype('uint8')
print "Stacking complete!"

print "Writing..."
#f = gzip.open(olabelsgz, "w")
f = open(olabels, "w")
cPickle.dump((labelsindex,labels), f)
#cPickle.dump((imagesindex,filtered,labels,contoursarray), f)
f.close()
os.system("gzip {}".format(olabels))
print("Saving labels complete! {}".format(time.ctime()))

(735, 1024, 1344)


/usr/local/lib/python2.7/site-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))


Threshold: 0.0600, Worm size: 1376256 Image covered: 100.00%
./Rep1_23-8-17/mlc metf/Original Image1.tiff --- 0.14%
Threshold: 0.0600, Worm size: 1376256 Image covered: 100.00%
./Rep1_23-8-17/crp metf/crp metf TF1.tiff --- 0.27%
Threshold: 0.0600, Worm size: 1376256 Image covered: 100.00%
./Rep1_23-8-17/mlc wo metf/mlc wo metf TF1.tiff --- 0.41%
Threshold: 0.0600, Worm size: 1376256 Image covered: 100.00%
./Rep1_23-8-17/csiR wo metf/cisR wo metf TF1.tiff --- 0.54%
Threshold: 0.0600, Worm size: 1376256 Image covered: 100.00%
./Rep1_23-8-17/csiR metf/csiR metf TF1.tiff --- 0.68%
Threshold: 0.0600, Worm size: 1376256 Image covered: 100.00%
./Rep1_23-8-17/argR metf/argR metf TF1.tiff --- 0.82%
Threshold: 0.0600, Worm size: 1376256 Image covered: 100.00%
./Rep1_23-8-17/ntrC wo metf/ntrC wo metf TF1.tiff --- 0.95%
Threshold: 0.0600, Worm size: 1376256 Image covered: 100.00%
./Rep1_23-8-17/fur metf/fur metf TF1.tiff --- 1.09%
Threshold: 0.0600, Worm size: 1376256 Image covered: 100.00%
./Rep1

## Save images

In [ ]:
if not loadimages:
    print("Saving images: {}".format(time.ctime()))
    print "Stacking data..."
    images=np.stack(imagesarray, axis=0)
    f = open(oimages, "w")
    #f = gzip.open(oimagesgz, "w")
    print "Writing..."
    cPickle.dump((imagesindex,images), f)
    f.close()
    os.system("gzip {}".format(oimages))
    print("Saving images complete!")

In [ ]:
print("Saving labels: {}".format(time.ctime()))
print "Stacking..."
#filtered=np.stack(filteredarray,axis=0)
labels=np.stack(labelsarray, axis=0).astype('uint8')
print "Stacking complete!"

print "Writing..."
#f = gzip.open(olabelsgz, "w")
f = open(olabels, "w")
cPickle.dump((labelsindex,labels), f)
#cPickle.dump((imagesindex,filtered,labels,contoursarray), f)
f.close()
os.system("gzip {}".format(olabels))
print("Saving labels complete! {}".format(time.ctime()))

### Load data

In [14]:
fileinfo=readTF('./Allfiles_annotated.csv')
genes=fileinfo.keys()
#genes=['OP50-C']

iimages="Images.pkl.gz"
ilabels="Segmentation_onlyprc.pkl.gz"

repflds=['Rep1_23-8-17','Rep2_24-8-17']

fd=3
od=5
plt.ioff()

./Allfiles_annotated.csv
Index([u'Replicate', u'Type', u'Gene', u'FileNo', u'FileInd',
       u'Replicate_folder', u'Folder', u'File', u'Max'],
      dtype='object')
Necessary headers found!


In [15]:
print time.ctime()

print "Loading images..."
imagesf = gzip.open(iimages,'rb')
imagesindex, images = cPickle.load(imagesf)
imagesf.close()
print "Loading finished!"
print time.ctime()

print "Loading labels..."
labelsf = gzip.open(ilabels,'rb')
labelsindex, labels = cPickle.load(labelsf)
labelsf.close()
print "Loading finished!"
print time.ctime()

Fri Mar  2 12:03:41 2018
Loading images...
Loading finished!
Fri Mar  2 12:04:39 2018
Loading labels...
Loading finished!
Fri Mar  2 12:05:23 2018


### Generate figures

In [ ]:
lbi=0
imi=0
for gene in genes:
    if gene=='OP50-C':
        nflds=6*2
    else:
        nflds=4*2
    gval=fileinfo[gene]
    gmax = gval['Max']
    print gmax
    
    fwidth=6*nflds
    fheight=5*gmax
    
    print nflds, gmax, fwidth, fheight
    fig, axes = plt.subplots(nrows=gmax, ncols=nflds, figsize=(fwidth,fheight), dpi=300)
    
    
    
    ofile="{}_segmentation_onlyprc.pdf".format(gene)
    
    print ofile
    col=0
    for rep in [1,2]:
        repval = gval[rep]
        repfld=repflds[rep-1]
        for tp in ['C','T']:
            tpval=repval[tp]
            
            for fld, fldvals in tpval.iteritems():
                print "Gene: {} Replicate: {} Type: {} Folder: {}".format(gene, rep, tp, fld)
                fldcontsel=fldvals['ByFIN']
                #print fldcontsel.keys()
                
                row=0
                for fli in fldcontsel.keys():
                    
                    fl=fldcontsel[fli]
                    
                    flfull="./{}/{}/{}".format(repfld,fld,fl)
                    print flfull #, os.path.isfile(flfull)
                    
                    #GetRGB data from a collapsed image
                    image_raw=images[imagesindex[flfull]]
                    if np.ndim(image_raw)==2:
                        v=image_raw
                        image=color.gray2rgb(image_raw)
                        imgrgb=image
                    else:
                        v=image[:,:,0]
                        image=image_raw
                        imgrgb=image
                    
                    #print image.shape

                    #imghsv = color.rgb2hsv(image)
                    
                    #v=imghsv[:,:,2]
                    #image_jet=jetimage(v,typeadjust=False)
                    dv = rank.median(v, disk(fd))
                    imagef = opening(dv, selem=disk(od))
                    
                    #Load labels
                    labeled_worms=labels[labelsindex[flfull]]
                    #imagef=filtered[lbid]
                    #contour=contours[lbid]
                    #contours = measure.find_contours(labeled_worms, 0.8)
                    
                    #Max value: 1
                    imgrgb = img_as_float(image)

                    
                    plotcontours(imagef,axes[row,col],ylabel=fl,title="{} - {}".format(rep,fld),plotcontour=False,plotlabels=False,white=False)
                    plotcontours(imgrgb,axes[row,col+1],labeled_worms,title="{}".format(fli),plotlabels=False)
                    row+=1         
                col+=2

    #fig.tight_layout()
    fig.savefig(ofile, bbox_inches='tight')
    plt.close(fig)
    
print "Finished!"
    
    

18
8 18 48 90
mlc_segmentation_onlyprc.pdf
Gene: mlc Replicate: 1 Type: C Folder: mlc wo metf
./Rep1_23-8-17/mlc wo metf/mlc wo metf TF1.tiff


/usr/local/lib/python2.7/site-packages/matplotlib/artist.py:210: MatplotlibDeprecationWarning: This has been deprecated in mpl 1.5, please use the
axes property.  A removal date has not been set.
  warnings.warn(_get_axes_msg, mplDeprecation, stacklevel=1)


./Rep1_23-8-17/mlc wo metf/mlc wo metf TF2.tiff
./Rep1_23-8-17/mlc wo metf/mlc wo metf TF3.tiff
./Rep1_23-8-17/mlc wo metf/mlc wo metf TF4.tiff
./Rep1_23-8-17/mlc wo metf/mlc wo metf TF5.tiff
./Rep1_23-8-17/mlc wo metf/mlc wo metf TF6.tiff
./Rep1_23-8-17/mlc wo metf/mlc wo metf TF7.tiff
./Rep1_23-8-17/mlc wo metf/mlc wo metf TF8.tiff
./Rep1_23-8-17/mlc wo metf/mlc wo metf TF9.tiff
./Rep1_23-8-17/mlc wo metf/mlc wo metf TF10.tiff
./Rep1_23-8-17/mlc wo metf/mlc wo metf TF11.tiff
./Rep1_23-8-17/mlc wo metf/mlc wo metf TF12.tiff
./Rep1_23-8-17/mlc wo metf/mlc wo metf TF13.tiff
./Rep1_23-8-17/mlc wo metf/mlc wo metf TF14.tiff
./Rep1_23-8-17/mlc wo metf/mlc wo metf TF15.tiff
./Rep1_23-8-17/mlc wo metf/mlc wo metf TF16.tiff
./Rep1_23-8-17/mlc wo metf/mlc wo metf TF17.tiff
Gene: mlc Replicate: 1 Type: T Folder: mlc metf
./Rep1_23-8-17/mlc metf/Original Image1.tiff
./Rep1_23-8-17/mlc metf/captured layer 32.tiff
./Rep1_23-8-17/mlc metf/captured layer 43.tiff
./Rep1_23-8-17/mlc metf/captured laye